# PhD Retreat 2024: Dynamical Systems Workshop
## Part A: Single machine infinite bus 

### The model
In the *single-machine-infinite-bus* model, grid dynamics stay unaffected by the one considered oscillator. The differential equation (for states $(\varphi, \omega)\in [0, 2\pi) \times \mathbb{R}$ in the co-rotating reference frame) reads
$$\begin{align*}
\frac{\mathrm{d}\varphi}{\mathrm{d}t} &= \omega \\
\frac{\mathrm{d}\omega}{\mathrm{d}t} &= P/M - \omega D/M - K/M \sin(\varphi)
\end{align*}$$
with locally produced/consumed power $P$, mass $M$, damping factor $D$, and coupling $K$. The cell below integrates the system from $t_0 = 0$ to $t = 2$. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.integrate import solve_ivp
from source.coupled_swing_systems import SmibSystem


# System parameters (intertia, damping, power injection, coupling)
M = 0.1; D = 0.05; P = 0.3; K = 1.0 # or  M = 0.02; D = 20; P = 5; K = 400.0



### Exercise 1a
1. Study the system behaviour for different combinations of parameters $(D, P, K)$
2. Try to find a combination for which the system does not oscillate.
3. Observe relaxation behaviour from different initial conditions $x_0 = (\varphi_0, \omega_0)\in [0, 2\pi) \times \mathbb{R}$ (e.g. $x_0 = ()$ or $()$). 

In [ ]:

# Simulation parameters
t_span = (0.0, 2.0) # time span for the simulation
t_eval = np.linspace(t_span[0], t_span[1], 3000)

# Stable Steady state value
u_eq = [np.arcsin(P / K), 0.]

# Problem definition
problem = SmibSystem(M, D, P, K) # define ode problem

# Simulate stable trajectory
u_init = [u_eq[0]+0.1, -.2] # initial conditions
solution = solve_ivp(problem, t_span, u_init, t_eval=t_eval)

# Plotting stable trajectory
fig, ax = plt.subplots()
ax.plot(solution.t, solution.y[1,:], label=r'$\varphi$')
fig.show()



### Exercise 2
Plot the phase portrait

In [ ]:
from source.utils import create_2Dphaseportrait

# Plot the phase portrait
phi_lims = (-np.pi, np.pi) + u_eq[0]; omega_lims = (-10, 10)
X, Y, U, V = create_2Dphaseportrait(problem, phi_lims, omega_lims, 0.0, N=50, normalize=False)



### Linear stability analysis 
[Eigenvalues and Eigenvectors in Python](https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter15.04-Eigenvalues-and-Eigenvectors-in-Python.html)